# Bostrom supply simulation
---

In [ ]:
import time
import itertools

# Standard libraries: https://docs.python.org/3/library/
import math

# Analysis and plotting modules
import pandas as pd
import matplotlib.pyplot as plt

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

from collections import Counter
from cadCAD import configs

# custom functions modules

from utils.partial_state_update_blocks import partial_state_update_blocks

from utils.plots import linear_plot, scatter_plot, df_preparator, plot_line_2_diff_y, plot_line_2_same_y

## Initial state

In [ ]:
sim_period = 10           # amount of years for simulating
timesteps_per_year = 365        # units per year(days`

In [ ]:
initial_state = {
    'boot_inflation': 0.10,
    'boot_supply': 1_000_000_000_000_000,
    'liquid_boot_amount': 200_000_000_000_000,
    'frozen_boot_amount': 700_000_000_000_000,
    'bonded_boot_amount': 100_000_000_000_000,
    'claimed_boot_amount': 0,
    'to_distribution_boot_amount':0,
    'agents_count': 250,
    'capitalization_per_agent': 1,
    'capitalization': 250 * 1,
    'delta_cyberlinks': 0,
    'validator_revenue': 0,
    'timestep_provision': 0,
    'gboot_price': 250 * 1 / 1_000_000,
    'cyberlinks': 0,
    'minted_amper_amount': 0,
    'minted_volt_amount': 0,
    'investmint_max_period': math.floor(timesteps_per_year / 4),
    'mint_rate_amper': 1,
    'mint_rate_volt': 1,
    'gpu_memory_usage': 0,
    'amper_volt_ratio': 0
}

## Params for simulating

In [ ]:
# vesting_speed = [3, 6, 12, 18, 24, 30, 36]
# unvesting_speed = [72, 60, 48, 36, 30, 24]

# parameter_sweep = list(itertools.product(vesting_speed, unvesting_speed))

# vesting_speed = [x[0] for x in parameter_sweep]
# unvesting_speed = [x[1] for x in parameter_sweep]

In [ ]:
system_params = {
    # timesteps params
    'timesteps_per_year': [timesteps_per_year], # units per year(minutes)
    
    # boot inflation params
    'boot_inflation_rate_change_annual': [0.07], # maximum inflation rate change per year
    'boot_inflation_min': [0.05], # minimum percent of the inflation
    'boot_inflation_max': [0.20], # maximum percent of the inflation
    'boot_bonded_share_target': [0.70], # desireable ratio between vested tokens and tokens supply
    
    # boot bonding amount params
    'boot_bonding_share': [0.70],
    
    # gift claim dynamics params
    'days_for_gift_activation': [30],
    'claimed_at_activation_share': [0.85],
    'days_for_gift_full_claim': [150],
    'boot_gift_amount_init': [700_000_000_000_000],
    'liquid_boot_supply_share': [0.25], # this param isn't used
    
    # understanding network effects params
    'agents_count_at_activation': [10_000],
    'start_capitalization_per_agent': [1],
    'capitalization_per_agent': [1],
    
    # predicting volts demand params
    'extra_links': [0],
    'guaranted_links': [0],
    
    # amper and volt minting params
    'base_halving_period_amper': [365 * 3 / 2],
    'base_halving_period_volt': [365 * 3 / 2],
    'mint_rate_amper_init': [1], # initial mint rate for Amperes
    'mint_rate_volt_init': [1], # initial mint rate for Voltes
    'base_investmint_amount_amper': [1_000_000_000], # initial cost for resource token in 1 cycle
    'base_investmint_amount_volt': [1_000_000_000], # initial cost for resource token in 1 cycle
    'base_investmint_period_amper': [math.floor(timesteps_per_year / 2)], # 3 month
    'base_investmint_period_volt': [math.floor(timesteps_per_year / 2)], # 3 month
    'investmint_max_period_init': [math.floor(timesteps_per_year / 3)], # 4 months
    'horizont_initial_period': [math.floor(timesteps_per_year)], # 6 months
    
    # Investments into infrastructure params
    'validator_commission': [0.1],
    'max_validator_count': [92],
}

In [ ]:
del configs[:]
experiment = Experiment()

sim_config = config_sim({
    'N': 1,
    'T': range(int(math.ceil(sim_period * timesteps_per_year))),
    'M': system_params
})

experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

In [ ]:
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

In [ ]:
start_time = time.time()
simulation_result = pd.DataFrame(raw_result)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()

In [ ]:
df = simulation_result.copy()

In [ ]:
# save local model file for power bi
# import pyarrow as pa
# import pyarrow.parquet as pq
# table = pa.Table.from_pandas(df)
# pq.write_table(table, 'simulation.parquet')

In [ ]:
plot_df = df_preparator(df)

In [ ]:
plot_df


## Boot suplly and inflation

In [ ]:
columns = ['liquid_boot_amount', 'frozen_boot_amount', 'bonded_boot_amount']
supply_df = plot_df[columns]
boot_inflation_df = plot_df['boot_inflation']

ax1 = supply_df.plot.area(linewidth=0, colormap="winter", xticks=range(0, 3650, 365), grid=True)

ax2 = ax1.twinx()
ax2.spines['right'].set_position(('axes', 1.0))
boot_inflation_df.plot.line(ax=ax2, figsize=(16,9), style={'boot_inflation': 'y'}, xticks=range(0, 3650, 365), grid=True)

In [ ]:
plot_df.plot(y='agents_count', figsize=(16, 9), xticks=range(0, 3650, 365), grid=True)

In [ ]:
# agents_count.plot.line(figsize=(16,9), logy=True, xticks=range(0, 3650, 365))
plot_line_2_diff_y(plot_df, 'capitalization_per_agent', 'capitalization', value_1_log=True)

In [ ]:
plot_line_2_same_y(plot_df, ['delta_cyberlinks', 'minted_volt_amount'])
# ax1 = df.plot(y=['delta_cyberlinks', 'minted_volt_amount'], figsize=(16, 9), xticks=range(0, 3650, 365), grid=True)
# ax2 = ax1.twinx()
# ax2.spines['right'].set_position(('axes', 1.0))
# plot_df.plot.line(ax=ax2, y='cyberlinks', figsize=(16, 9), xticks=range(0, 3650, 365), style={'cyberlinks': 'r'}, grid=True)


In [ ]:
plot_df.plot.line(x='timestep', y='cyberlinks', grid=True, xticks=range(0, 3650, 365), figsize=(16, 9))

In [ ]:
plot_line_2_diff_y(plot_df, 'gboot_price', 'validator_revenue')

In [ ]:
ax1 = df.plot(y=['mint_rate_amper', 'mint_rate_volt'], figsize=(16, 9), xticks=range(0, 3650, 365), grid=True)
ax2 = ax1.twinx()
ax2.spines['right'].set_position(('axes', 1.0))
plot_df.plot.line(ax=ax2, y='investmint_max_period', figsize=(16, 9), xticks=range(0, 3650, 365), style={'investmint_max_period': 'r'}, grid=True)

In [ ]:
ax1 = df.plot(y=['minted_amper_amount', 'cyberlinks'], figsize=(16, 9), xticks=range(0, 3650, 365), grid=True)
ax2 = ax1.twinx()
ax2.spines['right'].set_position(('axes', 1.0))
plot_df.plot.line(ax=ax2, y='gpu_memory_usage', figsize=(16, 9), xticks=range(0, 3650, 365), style={'gpu_memory_usage': 'r'}, grid=True)

In [ ]:
linear_plot(plot_df, 'amper_volt_ratio', render='sns')

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))